# Emissions Analysis for ST18 and ST14 Engines

In [7]:
import pandas as pd
# Set pandas to display all rows
pd.set_option('display.max_rows', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objs as go

In [27]:
# Load the datasets and combine them
df_st18 = pd.read_csv("GeneralInspectionReport ST18.csv")
df_st14 = pd.read_csv("GeneralInspectionReport ST14.csv")
df = pd.concat([df_st18, df_st14], ignore_index=True)

# Map machine numbers to engine tiers
tier_mapping = {
    'LD070': 'Tier 4', 'LD071': 'Tier 4', 'LD076': 'Tier 4', 'LD078': 'Tier 4', 'LD094': 'Tier 4',
    'LD079': 'Tier 5', 'LD083': 'Tier 5', 'LD084': 'Tier 5', 'LD082': 'Tier 5', 'LD086': 'Tier 5', 'LD085': 'Tier 5', 'LD072': 'Tier 5',
    'RB016': 'Tier 3A', 'LD210': 'Tier 3A'
}
df['Tier'] = df['Machine Number'].map(tier_mapping)

# Extract Condition and Pollutant
df[['Condition', 'Pollutant']] = df['Category'].str.extract(r'^(IDLE|HIGH IDLE|STALL|-)\s*(CO|NO2|NOX|NO|DPM mg/m3)?')

# Clean data
df_clean = df.dropna(subset=['Tier', 'Condition', 'Reading'])

def filter_outliers_iqr(group):
    Q1 = group['Reading'].quantile(0.25)
    Q3 = group['Reading'].quantile(0.75)
    IQR = Q3 - Q1
    # return group[(group['Reading'] >= Q1 - 1.5 * IQR) & (group['Reading'] <= Q3 + 1.5 * IQR)]
    return group[(group['Reading'] >= Q1) & (group['Reading'] <= Q3)]

df_filtered_iqr = df_clean.groupby(['Tier', 'Condition', 'Pollutant']).apply(filter_outliers_iqr).reset_index(drop=True)

# Remove rows with 'idle' or 'stall' in the Condition column (case-insensitive), but keep 'high idle'
df_filtered_iqr_no_idle_stall = df_filtered_iqr[~df_filtered_iqr['Condition'].str.lower().isin(['idle', 'stall'])]

# Compute average values post-filtering
average_values = df_filtered_iqr_no_idle_stall.groupby(['Tier', 'Condition', 'Pollutant'])['Reading'].mean().reset_index()
average_values.rename(columns={'Reading': 'Average Reading'}, inplace=True)

# Compute what the average values would look like if doubled
average_values['Doubled Average'] = average_values['Average Reading'] * 2

# Add Time Weighted Average and Short Term Exposure Limit columns
exposure_limits = pd.DataFrame({
    'Pollutant': ['CO', 'NO', 'NO2', 'NOX', 'DPM mg/m3'],
    'TWA': [30, 25, 3, '-', 0.1],
    'STEL': ['-', '-', 5, '-', '-']
})

average_values = average_values.merge(exposure_limits, on='Pollutant', how='left')
average_values


C:\Users\19065963\AppData\Local\Temp\ipykernel_37512\2145035774.py:27: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Tier,Condition,Pollutant,Average Reading,Doubled Average,TWA,STEL
0,Tier 3A,-,DPM mg/m3,310.000000,620.000000,0.1,-
1,Tier 3A,HIGH IDLE,CO,137.000000,274.000000,30,-
2,Tier 3A,HIGH IDLE,NO,144.000000,288.000000,25,-
3,Tier 3A,HIGH IDLE,NO2,0.700000,1.400000,3,5
4,Tier 3A,HIGH IDLE,NOX,146.500000,293.000000,-,-
5,Tier 4,-,DPM mg/m3,509.130435,1018.260870,0.1,-
6,Tier 4,HIGH IDLE,CO,7.250278,14.500556,30,-
7,Tier 4,HIGH IDLE,NO,37.708333,75.416667,25,-
8,Tier 4,HIGH IDLE,NO2,8.648000,17.296000,3,5
9,Tier 4,HIGH IDLE,NOX,48.758333,97.516667,-,-


In [28]:
# Convert CO, NO, NO2, NOX from PPM to mg/m3 and rename pollutant labels for all relevant columns
molar_weights = {'CO': 28.01, 'NO': 30.01, 'NO2': 46.01, 'NOX': 46.01}
def ppm_to_mgm3(ppm, molwt):
    return ppm * molwt / 24.465

for idx, row in average_values.iterrows():
    pol = row['Pollutant']
    if pol in molar_weights:
        # Convert Average Reading
        mgm3 = ppm_to_mgm3(row['Average Reading'], molar_weights[pol])
        average_values.at[idx, 'Average Reading'] = mgm3
        # Convert Doubled Average
        if 'Doubled Average' in average_values.columns:
            mgm3_doubled = ppm_to_mgm3(row['Doubled Average'], molar_weights[pol])
            average_values.at[idx, 'Doubled Average'] = mgm3_doubled
        # Convert TWA and STEL if present and not '-'
        if 'TWA' in average_values.columns and row['TWA'] != '-':
            average_values.at[idx, 'TWA'] = ppm_to_mgm3(row['TWA'], molar_weights[pol])
        if 'STEL' in average_values.columns and row['STEL'] != '-':
            average_values.at[idx, 'STEL'] = ppm_to_mgm3(row['STEL'], molar_weights[pol])
        # Rename pollutant
        average_values.at[idx, 'Pollutant'] = f"{pol} mg/m3"


# Remove NOX rows (including any renamed to 'NOX mg/m3')
average_values = average_values[~average_values['Pollutant'].isin(['NOX mg/m3'])].reset_index(drop=True)
average_values

,Tier,Condition,Pollutant,Average Reading,Doubled Average,TWA,STEL
0,Tier 3A,-,DPM mg/m3,310.000000,620.000000,0.1,-
1,Tier 3A,HIGH IDLE,CO mg/m3,156.851420,313.702841,34.347026,-
2,Tier 3A,HIGH IDLE,NO mg/m3,176.637646,353.275291,30.666258,-
3,Tier 3A,HIGH IDLE,NO2 mg/m3,1.316452,2.632904,5.641937,9.403229
4,Tier 4,-,DPM mg/m3,509.130435,1018.260870,0.1,-
5,Tier 4,HIGH IDLE,CO mg/m3,8.300849,16.601699,34.347026,-
6,Tier 4,HIGH IDLE,NO mg/m3,46.254939,92.509878,30.666258,-
7,Tier 4,HIGH IDLE,NO2 mg/m3,16.263825,32.527650,5.641937,9.403229
8,Tier 5,-,DPM mg/m3,359.333333,718.666667,0.1,-
9,Tier 5,HIGH IDLE,CO mg/m3,12.763525,25.527049,34.347026,-


In [32]:
# Copy the table to work with
interactive_df = average_values.copy()

# Interactive widgets
def create_interactive_table():
    exhaust_flow = widgets.FloatSlider(value=0.387, min=0, max=5, step=0.001, description='Exhaust G (m³/s)')
    Q = widgets.FloatSlider(value=20, min=0, max=50, step=0.1, description='Vent Q (m³/s)')
    V = widgets.FloatSlider(value=8616.63, min=0, max=10000, step=1, description='Volume V (m³)')
    C0 = widgets.FloatSlider(value=0, min=0, max=1000, step=0.1, description='Initial C₀ (mg/m³)')
    t = widgets.FloatSlider(value=10, min=0, max=120, step=1, description='Time t (min)')

    tau_label = widgets.HTML()
    # Dropdowns for Tier and Pollutant
    tier_options = sorted(interactive_df['Tier'].unique())
    pollutant_options = sorted(interactive_df['Pollutant'].unique())
    tier_dropdown = widgets.Dropdown(options=tier_options, description='Tier:')
    pollutant_dropdown = widgets.Dropdown(options=pollutant_options, description='Pollutant:')

    def update_table(*args):
        df = interactive_df.copy()
        C_ss_list = []
        C_t_list = []
        for idx, row in df.iterrows():
            avg = row['Average Reading']
            conc_mgm3 = avg  # Use value as-is
            # Mass flow rate for one tailpipe (mg/s)
            mass_flow_1 = conc_mgm3 * exhaust_flow.value
            G = 2 * mass_flow_1  # two tailpipes
            C_ss = G / Q.value if Q.value else ''
            tau = V.value / Q.value if Q.value else float('nan')
            t_sec = t.value * 60
            if Q.value and V.value:
                C_t = (G/Q.value) + (C0.value - (G/Q.value)) * np.exp(-Q.value/V.value * t_sec)
            else:
                C_t = float('nan')
            C_ss_list.append(C_ss)
            C_t_list.append(C_t)
        df['C_SS (mg/m³)'] = C_ss_list
        df['C(t) (mg/m³)'] = C_t_list
        tau_label.value = f"<b>Tau (τ) = V / Q = {V.value:.2f} / {Q.value:.2f} = {tau:.2f} s</b>" if Q.value else "<b>Tau (τ) is undefined (Q=0)</b>"
        with out:
            clear_output(wait=True)
            display(widgets.HBox([tau_label, t]))
            display(df)
            # Plot for selected Tier and Pollutant
            selected = df[(df['Tier'] == tier_dropdown.value) & (df['Pollutant'] == pollutant_dropdown.value)]
            if selected.empty:
                print("No data for selected Tier and Pollutant.")
                return
            row = selected.iloc[0]
            avg = row['Average Reading']
            conc_mgm3 = avg
            mass_flow_1 = conc_mgm3 * exhaust_flow.value
            G = 2 * mass_flow_1
            Qv = Q.value
            Vv = V.value
            C0v = C0.value
            t_vals = np.linspace(0, 120*60, 200)
            C_ss = G / Qv if Qv else 0
            C_t_curve = (G/Qv) + (C0v - (G/Qv)) * np.exp(-Qv/Vv * t_vals) if Qv and Vv else np.zeros_like(t_vals)
            t_current = t.value * 60
            C_t_current = (G/Qv) + (C0v - (G/Qv)) * np.exp(-Qv/Vv * t_current) if Qv and Vv else 0
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=t_vals/60, y=C_t_curve, mode='lines', name='C(t)'))
            fig.add_hline(y=C_ss, line_dash='dash', line_color='red', annotation_text='C_SS')
            fig.add_trace(go.Scatter(x=[t.value], y=[C_t_current], mode='markers+text',
                                     marker=dict(color='blue', size=10),
                                     text=[f"t={t.value} min<br>C(t)={C_t_current:.2f} mg/m³"],
                                     textposition="top center",
                                     name='Current t'))
            fig.update_layout(title=f'Ventilation Dilution for {tier_dropdown.value}, {pollutant_dropdown.value}',
                              xaxis_title='Time (min)', yaxis_title='Concentration (mg/m³)')
            fig.show()

    out = widgets.Output()
    for w in [exhaust_flow, Q, V, C0, t, tier_dropdown, pollutant_dropdown]:
        w.observe(update_table, names='value')
    display(widgets.HBox([exhaust_flow, Q, V, C0, tier_dropdown, pollutant_dropdown]))
    update_table()
    display(out)

create_interactive_table()

Output()